# DeepESN <br/>
https://github.com/gallicch/DeepRC-TF

In [1]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split

from DeepRC import *

In [2]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [3]:
df = pd.read_csv("data/train.csv")
df['comment_text'] = df['comment_text'].map(lambda com : clean_text(com))

categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
num_classes = len(categories)
train, valid = train_test_split(df, random_state=42, test_size=0.33, shuffle=True)

In [4]:
X_train = train.comment_text
y_train = train[categories]
X_valid = valid.comment_text
y_valid = valid[categories]

tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_matrix(X_train, mode='tfidf')
X_valid = tokenizer.texts_to_matrix(X_valid, mode='tfidf')

X_train = tf.expand_dims(X_train, axis=2)
X_valid = tf.expand_dims(X_valid, axis=2)

y_train = y_train.values
y_valid = y_valid.values

print(X_train.shape,X_valid.shape)
print(y_train.shape,y_valid.shape)

(106912, 500, 1) (52659, 500, 1)
(106912, 6) (52659, 6)


In [ ]:
reservoir_units = 100
batch_size = 32
lr = 0.001

inputs = Input(shape=(reservoir_units,))
outputs = Dense(num_classes, activation='softmax')(inputs)
readout = Model(inputs, outputs)
readout.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])

for b in range(0, len(X_train), batch_size):
    X = X_train[b:b+batch_size]
    y = y_train[b:b+batch_size]
    
    reservoir = SimpleDeepReservoirLayer(units=reservoir_units, layers=5)(X)
    print(reservoir.shape,y.shape)
    readout.fit(reservoir, y, epochs=1, verbose=1)
    